# GOFAST speed estimation

In [83]:
import scipy as sc
import numpy as np
import itertools
from typing import List, Dict, Tuple, Any


def ft_to_m(dist_ft: float) -> float:
    return dist_ft * 0.3048


def m_to_ft(dist_m: float) -> float:
    return dist_m / 0.3048


def deg_to_rad(angle: float) -> float:
    return (np.pi / 180.) * angle


def ms_to_kmh(speed):
    '''
    args
        speed - speed in units [m / s]
    return
        speed - speed in units [km / h]
    '''
    return (speed * 3.6)


def speed(distance, time):
    return (distance / time)


def speed_m_s_kmh(distance: float, time: float) -> float:
    '''
    args
        distance - distance [m]
        time     - time [s]
    return
        velocity - [km / h]
    '''
    return (distance / 1000.) / (time / 3600.)


def angles_to_xy_coord(init_point_xy, plane_alt: float, 
                       down_angle: List,
                       left_angle: List) -> List:
    '''
    Calculate coordinates from two arrays of angles.
    Both tuples must have equal no of elements.
    
    args
        init_point_xy - initial point [unused]
        plane_alt     - altitude of plane [m]
        down_angle    - list of angles (can be negative, relative to plane) [deg]
        left_angle    - list of angles [deg]
    return
        xy_coord      - list of (x, y) coordinates [[x_1, y_1], [x_2, y_2], . . .]
    '''
    xy_coord = []
    
    for (down, left) in zip(down_angle, left_angle):
        down_rad_pos = deg_to_rad(np.abs(down))
        d2 = plane_alt / np.tan(down_rad_pos)
        #
        left_rad_pos = deg_to_rad(np.abs(left))
        x = d2 * np.cos(left_rad_pos)
        #
        y = d2 * np.sin(left_rad_pos)
        #
        xy_coord.append([x, y])
        
    return xy_coord


def distances_from_coords(coords: List) -> List:
    '''
    args
        coords    - coordinate pairs
    return
        distances - length is (len(coords) - 1)
    '''
    distances = []
    xy_vecs = []
    
    for i in range(len(coords) - 1):
        xy_vecs.append([(coords[i + 1][0] - coords[i][0]), (coords[i + 1][1] - coords[i][1])])
        distances.append(np.sqrt((xy_vecs[i][0])**2 + (xy_vecs[i][1])**2))
        
    return xy_vecs, distances


# main()

# time [s]
time = [12,   13,   14,   15,   16,   17]

# ORIGINAL RAW DATA
# plane_down_angle = [-26, -26, -27, -27, -27, -28]

# AVERAGED DATA
plane_down_angle = [-26., -26.4, -26.8, -27.2, -27.6, -28.]

# ORIGINAL DATA
plane_left_angle = [ 43.,  44.,  45.,  46.,  47.,  48.]

plane_init_altitude_ft = 25000.
conv_ft_to_m = 0.3048
plane_init_altitude_m = plane_init_altitude_ft * conv_ft_to_m

conv_km_to_nm = 0.539957

initial_down_angle = 26.

# print header
print("-----------------------------------------------------------------------------")
print("DATA from youtube video GOFAST")
print("-----------------------------------------------------------------------------")
print("Time period (s)           : 12 - 17")
print("Down angle / tilt (deg)   : (-26) - (-28)")
print("Left angle (deg)          : 43 - 48")
print()
print("Assumption: use the approximation that in this short period of time") 
print("            plane moves straight, ie does not turn.")
print()
print("NOTE      : The following calculations are in the REST FRAME of the plane.")
print("-----------------------------------------------------------------------------")
print()
print()

for i in range(5):
    plane_alt_m = plane_init_altitude_m - (i * 1000.)
    initial_distance_obj_m = plane_alt_m / np.tan(deg_to_rad(initial_down_angle))
    initial_distance_obj_nm = (initial_distance_obj_m / 1000.) * conv_km_to_nm
    initial_distance_obj_ft = m_to_ft(initial_distance_obj_m)
    
    print("-------")
    print("CASE {}".format(i))
    print("-------\n")
    print("(plane altitude = {:.0f} m | {:.0f} ft) => (distance / range = {:.0f} m | {:.0f} ft | {:.2f} nm)".format(plane_alt_m, 
                                                                                          m_to_ft(plane_alt_m),
                                                                                          initial_distance_obj_m,
                                                                                          initial_distance_obj_ft,
                                                                                          initial_distance_obj_nm))
    print()
    
    coords = angles_to_xy_coord(0., plane_alt_m, plane_down_angle, plane_left_angle)

    # raw coordinates [m]
    print("coordinates [[x, y]] = [", end='')
    print(", ".join('[{:.0f}, {:.0f}]'.format(x, y) for x, y in coords), end='')
    print("]")
    #print()

    # TODO save velocity vector
    #velocity = []
    
    array_len = len(time)
    xy_vecs, distances = distances_from_coords(coords)
    
    # DEBUG
    #print("xy_vecs = {}".format(xy_vecs))
    #print()

    # xy vectors
    print("velocity components [v_x, v_y] = [", end='')
    print(", ".join('[{:.0f}, {:.0f}]'.format(x, y) for x, y in xy_vecs), end='')
    print("]")
    print()    
    
    # distances
    print("distances between points [m] = [", end='')
    print(", ".join('{:.0f}'.format(k) for k in distances), end='')
    print("]")
    print()

    speed_ms = []
    for d in distances:
        speed_ms.append(speed(d, 1.))
    
    speed_kmh = []
    for v in speed_ms:
        speed_kmh.append(ms_to_kmh(v))
    
    # banner
    print("Calculate speeds =>")
    print()
    
    # speed m/s
    print("speed [m / s] = [", end='')
    print(", ".join('{:.0f}'.format(k) for k in speed_ms), end='')
    print("]")
    #print()

    # speed km/h
    print("speed [km / h] = [", end='')
    print(", ".join('{:.0f}'.format(k) for k in speed_kmh), end='')
    print("]")
    #print()

    # speed knots = nm/h
    print("speed [knots] = [", end='')
    print(", ".join('{:.0f}'.format((k * conv_km_to_nm)) for k in speed_kmh), end='')
    print("]")
    print()
    print("-----------------------------------------------------------------------------")
    print()


-----------------------------------------------------------------------------
DATA from youtube video GOFAST
-----------------------------------------------------------------------------
Time period (s)           : 12 - 17
Down angle / tilt (deg)   : (-26) - (-28)
Left angle (deg)          : 43 - 48

Assumption: use the approximation that in this short period of time
            plane moves straight, ie does not turn.

NOTE      : The following calculations are in the REST FRAME of the plane.
-----------------------------------------------------------------------------


-------
CASE 0
-------

(plane altitude = 7620 m | 25000 ft) => (distance / range = 15623 m | 51258 ft | 8.44 nm)

coordinates [[x, y]] = [[11426, 10655], [11042, 10663], [10667, 10667], [10300, 10666], [9941, 10660], [9589, 10650]]
velocity components [v_x, v_y] = [[-384, 8], [-375, 3], [-367, -1], [-359, -6], [-351, -10]]

distances between points [m] = [384, 375, 367, 359, 351]

Calculate speeds =>

speed [m / s] = 